# Hierarchical Clustering of Countries Economic Development

## Introduction
The aim of this notebook is to determine which hierarchical clustering algorithm is optimal for the task of grouping the financial growth of 113 countries over the past 58 years. The whole process is documented step by step.

### Imports

In [1]:
import numpy as np

from tslearn.clustering import silhouette_score

from Utils import DataUtils, PreProcessing, TimeSeriesUtils, VisualUtils

from Clustering import Outliers, HierarchicalClustering

### Data
For each country, three different time series of data are considered; the annual evolution of its GDP per capita, the annual evolution of its population and the annual evolution of the bilateral exchange rate of its currency against the US Dollar. While there is (partially) available data for the GDP and population evolution all the way from 1900, data about currency exchange rates is only available since 1960. For all three time series, the latest data concerns 2017. In order to stabilize the dataset, all GDP and population data before 1960 is discarded.

Because the GDP time series present very high variance, the logarithm of the series is taken instead to smoothen the results.

The data for each time series is loaded from a `.csv` file to create a $113 \times T$ matrix, where $T$ is the amount of years each time series is available for.

In [2]:
n = 113
T_gdp = 118
T_pop = 118
T_currency = 58

start_year = 1960

names_path = "Data/names.txt"
gdp_path = "Data/yp_raw.csv"
population_path = "Data/pop_raw.csv"
currency_path = "Data/currency.csv"
map_path = "Data/Map/ne_110m_admin_0_countries.shp"

In [3]:
names, gdp, pop, currency, map = DataUtils.load_clustering_data(
    names_path, gdp_path, population_path, currency_path, map_path, n, T_gdp, T_pop, T_currency
)
gdp_data = np.log(gdp[:, -T_currency:])
pop_data = pop[:, -T_currency:]

## Data Preparation

Since the dataset consists of three different time series for each country, it would be irrational to construct feature vectors by concatenating the time series. Instead, Principal Component Analysis is performed on each time series to determine the components that capture the most variance. By specifying the variance, the PCA  implementation from sklearn automatically determines the number of components for each dataset.

Once the principal components for each series are computed and concatenated, standard scaling is performed to normalize the dataset.

One complication that arises for currency exchange rate data when it comes to countries of former Yugoslavia and USSR. For these countries (Serbia, Croatia, Bosnia and Herzegovina and Russia respectively) no data is available until 1990. To tackle this problem, a nearest neighbor imputer is used to fill the empty values. While this is not indicative of the actual values for that period, it ensures the consistency of the dataset. This imputation is performed before the PCA algorithm.

In [4]:
variance = 0.95
df, scaled_df = PreProcessing.preprocess_pca(
    names, gdp_data, pop_data, currency, 0.95
)

## Outlier Detection

In order to detect and eliminate outliers from the dataset (i.e. countries that behave in a way dissimilar to all others), the DBSCAN clustering algorithm is employed. In order to tune the `eps` and `min_samples` parameters, the following heuristics are used:
- The `eps` parameter is set to the point of maximum curvature in the $k$-Distance Graph of the dataset.
- The `min_samples` parameter is set to twice the number of features in the dataset.

The first two tables displayed by the following python cell summarize the data in the dataset before and after outlier detection. The third table lists the countries that have been labeled as outliers. These countries are discarded from the dataset.

The plot displayed presents the $k$-Distance Graph of the dataset, for $k=2$.

In [ ]:
without_outliers, outliers, without_outliers_countries, outliers_countries = Outliers.remove_outliers_dbscan(
    names, scaled_df, True
)

## Hierarchical Clustering Algorithms

In this section, hierarchical clustering of the countries is performed with four different linkage methods, namely complete, average, single and ward linkage.

For each linkage method, the distance threshold for splitting clusters is determined by MATLAB(TM). Namely the threshold is set to $0.7 * \text{max}(\text{distance})$. The number of clusters, the distribution of values and the silhoutte score are printed for each algorithm. Furthermore, the hierarchy is printed in the form of a dendogram to visualize the clusters.

### Complete Linkage

In [ ]:
complete_labels, complete_score = HierarchicalClustering.hierarchical_clustering(
    without_outliers, 'complete', True
)

### Average Linkage

In [ ]:
average_labels, average_score = HierarchicalClustering.hierarchical_clustering(
    without_outliers, 'average', True
)

### Single Linkage

In [ ]:
single_labels, single_score = HierarchicalClustering.hierarchical_clustering(
    without_outliers, 'single', True
)

### Ward Linkage

In [ ]:
ward_labels, ward_score = HierarchicalClustering.hierarchical_clustering(
    without_outliers, 'ward', True
)

## Conclusion

In conclusion, it seems like ward linkage is better suited for this problem, as it produces more balanced clusters and achieves the highest silhoutte score. This applies to 108 countries, since China, Indonesia, India, Iran and Vietnam are considered outliers. We now give a visualization of the proposed clusters, where the explicit centroids have been calculated using DTW Barycenter Averaging

In [10]:
df_gdp, scaled_df_gdp, scaled_data_gdp = PreProcessing.preprocess_onlyGDP(names, gdp_data, start_year, T_currency)
ward_labels -= 1

In [11]:
df_no_outliers = df_gdp.loc[without_outliers_countries]
df_outliers = df_gdp.loc[outliers_countries]

In [ ]:
score = silhouette_score(df_no_outliers, ward_labels)
# cluster_centers = TimeSeriesUtils.cluster_centroids(df_no_outliers.values, 3, ward_labels, T_currency)
clusters = VisualUtils.show_clustering(
    without_outliers_countries,
    3,
    df_no_outliers.values,
    None,
    ward_labels,
    score,
    3,
    1,
    start_year,
    T_currency,
    "Hierachical Clustering with Ward Linkage"
)

Even though we can clearly detect intracluster similarities, the clusters are not well seperated. This is well depected in the silhoutte score of the algorithm to (when taken against the scaled data and not the PCA-reduced dataset).

At last, we take a look on the outliers and the centroids, to see if any of the outliers would fit into the existing clusters:

In [13]:
# VisualUtils.plot_centroids_outliers(
#     cluster_centers, 
#     df_outliers.values, 
#     start_year, 
#     T_currency, 
#     "Centroids and Outliers for hierarchical clustering"
# )

We can see that some ot the outliers are very simiar to the centroids,, even though they provide significant improvement on the performance on the PCA-reduced data.